In [1]:
from __future__ import print_function, division, absolute_import

from copy import copy
# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

from cobrame.core.component import (Constraint, ProcessedProtein, Complex,
                                    TranslatedGene, TranscribedGene, Metabolite, Ribosome,
                                    GenericComponent,GenerictRNA,RNAP)

print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
# Define Models
model_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
model = cobra.io.load_json_model(model_directory)

In [3]:
with open('./me_models/iBS.pickle', 'rb') as f:
    me = pickle.load(f)

In [5]:
print_reactions_of_met(me,'mql7_c',1)

( AMMQT7_FWD_BSU22750-MONOMER 0 999999.0 ) 	 2dmmq7_c + 4.71514146737046e-6*mu BSU22750-MONOMER + amet_c + nadph_c --> ahcys_c + mql7_c + nadp_c
( NADH4_FWD_BSU32100-MONOMER 0 999999.0 ) 	 8.26913491691037e-5*mu BSU32100-MONOMER + h_c + mqn7_c + nadh_c --> mql7_c + nad_c
( SUCD5_BS_FWD_CPLX8J2-15 0 999999.0 ) 	 1.28067513227184e-6*mu CPLX8J2-15 + fadh2_c + 2.0 h_e + mqn7_c --> fad_c + 2.0 h_c + mql7_c
( G3PD4_FWD_BSU09300-MONOMER 0 999999.0 ) 	 2.04130467975336e-6*mu BSU09300-MONOMER + glyc3p_c + mqn7_c --> dhap_c + mql7_c
( NADH4_FWD_BSU12290-MONOMER_mod_nad 0 999999.0 ) 	 4.27350427350427e-6*mu BSU12290-MONOMER_mod_nad + h_c + mqn7_c + nadh_c --> mql7_c + nad_c
( NADH4_FWD_BSU32200-MONOMER 0 999999.0 ) 	 4.27350427350427e-6*mu BSU32200-MONOMER + h_c + mqn7_c + nadh_c --> mql7_c + nad_c


In [26]:
for key,value in model.reactions.BIOMASS_BS_10.metabolites.items():
    print(key,value)

10fthf_c -0.000367
adp_c 104.997414
ala__L_c -0.266902
amp_c -0.00467
arg__L_c -0.193021
asn__L_c -0.147987
asp__L_c -0.148014
atp_c -105.053483
ca2_c -0.003205
cdlp_BS_c -5e-06
cdp_c -0.000251
cmp_c -0.001042
ctp_c -0.038902
cys__L_c -0.05699
d12dg_BS_c -0.00011
datp_c -0.022982
dctp_c -0.01738
dgtp_c -0.017398
dttp_c -0.022899
fe3_c -0.00345
gdp_c -0.00018
gln__L_c -0.260335
glu__L_c -0.260378
gly_c -0.408288
gmp_c -0.000503
gtca1_45_BS_c -0.003624
gtca2_45_BS_c -0.002347
gtca3_45_BS_c -0.001819
gtp_c -0.062667
h2o_c -105.0
h_c 105.0
his__L_c -0.081739
ile__L_c -0.269905
k_c -0.706312
leu__L_c -0.346445
lipo1_24_BS_c -7e-06
lipo2_24_BS_c -6e-06
lipo3_24_BS_c -1.8e-05
lipo4_24_BS_c -1.5e-05
lys__L_c -0.323093
lysylpgly_BS_c -2.2e-05
m12dg_BS_c -8.6e-05
met__L_c -0.113326
mg2_c -0.101714
mql7_c -0.000266
nad_c -0.016164
nadp_c -0.000934
nadph_c -0.000216
peptido_BS_c -0.101817
pgly_BS_c -0.000176
phe__L_c -0.175939
pi_c 104.985613
ppi_c -0.000918
pro__L_c -0.160642
psetha_BS_c -0.00056

In [6]:
model.metabolites.get_by_id('2dmmq7_c').name

'2 Demethylmenaquinone 7 C45H62O2'

In [25]:
met

<Metabolite val__L_c at 0x7fc724dfe128>